In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")
from YOLOfunctional import BasicBlock,upsample,decode_trt,load_weights_darknet,nms_layer
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,ZeroPadding2D,MaxPool2D, LeakyReLU,UpSampling2D,Concatenate


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

from skimage.io import imread,imshow
from skimage.transform import resize
import time
import os

from pycocotools.coco import COCO

In [3]:
#anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]
anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]

num_classes = 0
last_filters = 3*(5+num_classes)
x = inputs = Input([416, 416, 3])
yolo1 = BasicBlock(x,num_filters = 16, kernel_size = 3,name="BasicBlock1")
yolo1 = BasicBlock(yolo1,num_filters = 32, kernel_size = 3,name="BasicBlock2")
yolo1 = BasicBlock(yolo1,num_filters = 64, kernel_size = 3,name="BasicBlock3")
yolo1 =  BasicBlock(yolo1,num_filters = 128, kernel_size = 3,name="BasicBlock4")
yolo1,root = BasicBlock(yolo1,num_filters = 256, kernel_size = 3, root = True,name="BasicBlock5")
yolo1 = BasicBlock(yolo1,num_filters = 512, kernel_size = 3,max_pool_stride=1,name="BasicBlock6")
yolo1 = BasicBlock(yolo1,num_filters = 1024, kernel_size = 3,max_pooling=False,name="BasicBlock7")
yolo1_branch =  BasicBlock(yolo1,num_filters = 256, kernel_size = 1,max_pooling=False,name="BasicBlock8")
yolo1 = BasicBlock(yolo1_branch,num_filters = 512, kernel_size = 3,max_pooling=False,name="BasicBlock9")
yolo1 = BasicBlock(yolo1,num_filters = last_filters, kernel_size = 1, batch_norm =False, max_pooling=False, act = None,name="FinalBlock1")

yolo2 = BasicBlock(yolo1_branch,num_filters = 128,kernel_size = 1,max_pooling = False,name="BasicBlock11")
yolo2 = upsample(yolo2)
yolo2 = Concatenate(axis=-1,name="Concatenate")([yolo2,root])
yolo2 = BasicBlock(yolo2,num_filters = 256,kernel_size = 3,max_pooling = False,name="BasicBlock12")
yolo2 = BasicBlock(yolo2,num_filters = last_filters, kernel_size = 1, batch_norm =False, max_pooling=False, act = None,name="FinalBlock2")

final_output_1 = decode_trt(yolo1,[1/13],anchors[:3],num_classes=num_classes)
final_output_2 = decode_trt(yolo2,[1/26],anchors[3:],num_classes=num_classes)

#final_output =nms_layer((final_output_1,final_output_2))

(None, None, None, 3, 5)
(None, None, None, 3, 5)


In [4]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [5]:
model = tf.keras.Model(inputs, (final_output_1, final_output_2))

In [6]:
#model.save_weights(current_directory+'/weights_functional/original_model')

In [7]:
#model = tf.keras.Model(inputs, (final_output))

In [8]:
aux = 0
nombres =[]
for index,i in enumerate(model.layers[1:45]):
    if index<4:
        #print(i.name)
        nombres.append((i.name+"_0").split("_"))
        #print((i.name+"_0").split("_"))
    else:
        #print(i.name)
        #print((i.name).split("_"))   
        nombres.append((i.name).split("_"))

nombres_definitivo = []
for i in nombres:
    try: 
        int(i[-1])
        nombres_definitivo.append(i)
    except:
        pass
    
nombres_definitivo.sort(key=lambda x: int(x[-1]))

nombres_definitivo_oficial = []

for k in nombres_definitivo:
    if k[-1]=='0':
        nombres_definitivo_oficial.append("_".join(k[:-1]))
    else:
        nombres_definitivo_oficial.append("_".join(k))

In [9]:
nombres_definitivo_oficial

['conv2d',
 'batch_normalization',
 'leaky_re_lu',
 'max_pooling2d',
 'conv2d_1',
 'batch_normalization_1',
 'leaky_re_lu_1',
 'max_pooling2d_1',
 'conv2d_2',
 'batch_normalization_2',
 'leaky_re_lu_2',
 'max_pooling2d_2',
 'conv2d_3',
 'batch_normalization_3',
 'leaky_re_lu_3',
 'max_pooling2d_3',
 'conv2d_4',
 'batch_normalization_4',
 'leaky_re_lu_4',
 'max_pooling2d_4',
 'conv2d_5',
 'batch_normalization_5',
 'leaky_re_lu_5',
 'max_pooling2d_5',
 'conv2d_6',
 'batch_normalization_6',
 'leaky_re_lu_6',
 'conv2d_7',
 'batch_normalization_7',
 'leaky_re_lu_7',
 'conv2d_8',
 'batch_normalization_8',
 'leaky_re_lu_8',
 'batch_normalization_9',
 'leaky_re_lu_9',
 'conv2d_9',
 'conv2d_10',
 'batch_normalization_10',
 'leaky_re_lu_10',
 'conv2d_11',
 'conv2d_12']

In [10]:
lista_definitiva = nombres_definitivo_oficial[:33] +['conv2d_9','conv2d_10','batch_normalization_9',
 'leaky_re_lu_9','conv2d_11','batch_normalization_10',
 'leaky_re_lu_10','conv2d_12']

In [11]:
lista_definitiva

['conv2d',
 'batch_normalization',
 'leaky_re_lu',
 'max_pooling2d',
 'conv2d_1',
 'batch_normalization_1',
 'leaky_re_lu_1',
 'max_pooling2d_1',
 'conv2d_2',
 'batch_normalization_2',
 'leaky_re_lu_2',
 'max_pooling2d_2',
 'conv2d_3',
 'batch_normalization_3',
 'leaky_re_lu_3',
 'max_pooling2d_3',
 'conv2d_4',
 'batch_normalization_4',
 'leaky_re_lu_4',
 'max_pooling2d_4',
 'conv2d_5',
 'batch_normalization_5',
 'leaky_re_lu_5',
 'max_pooling2d_5',
 'conv2d_6',
 'batch_normalization_6',
 'leaky_re_lu_6',
 'conv2d_7',
 'batch_normalization_7',
 'leaky_re_lu_7',
 'conv2d_8',
 'batch_normalization_8',
 'leaky_re_lu_8',
 'conv2d_9',
 'conv2d_10',
 'batch_normalization_9',
 'leaky_re_lu_9',
 'conv2d_11',
 'batch_normalization_10',
 'leaky_re_lu_10',
 'conv2d_12']

In [12]:
nombres_definitivo_oficial[32]

'leaky_re_lu_8'

In [13]:
model.get_layer("conv2d")

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 416, 416, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 416, 416, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [15]:
load_weights_darknet(model,'yolov3-tiny.weights',num_classes = num_classes)

conv2d
1
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
conv2d
batch_normalization
496
batch_normalization
4
496
leaky_re_lu
0
496
max_pooling2d
0
496
conv2d_1
1
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
conv2d_1
batch_normalization_1
5232
batch_normalization_1
4
5232
leaky_re_lu_1
0
5232
max_pooling2d_1
0
5232
conv2d_2
1
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
conv2d_2
batch_normalization_2
23920
batch_normalization_2
4
23920
leaky_re_lu_2
0
23920
max_pooling2d_2
0
23920
conv2d_3
1
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
conv2d_3
batch_normalization_3
98160
batch_normalization_3
4
98160
leaky_re_lu_3
0
98160
max_pooling2d_3
0
98160
conv2d_4
1
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
conv2d_4
batch_normalization_4
394096
batch_normalization_4
4
394096
leaky_re_lu_4
0
394096
max_pooling2d_4
0
394096
conv2d_5
1
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
conv2d_5
batch_normalization_5


8858734

In [16]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = tf.image.resize(img, (416, 416))/255

In [17]:
boxes = model(img)

In [18]:
boxes[0][0]

<tf.Tensor: shape=(1, 507, 4), dtype=float32, numpy=
array([[[0.0216936 , 0.06280277, 0.02019923, 0.05866241],
        [0.04394209, 0.04088125, 0.0294633 , 0.48675555],
        [0.05738147, 0.07290378, 0.27640754, 0.5104138 ],
        ...,
        [0.94484854, 0.98049814, 0.01925139, 0.18042621],
        [0.976819  , 0.9572642 , 0.01888947, 0.17309894],
        [0.97190577, 0.99066776, 0.31140846, 0.78413236]]], dtype=float32)>

In [19]:
nms_layer(boxes)

CombinedNonMaxSuppression(nmsed_boxes=<tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.1358022 , 0.34736437, 0.15610851, 0.60431856],
        [0.09304193, 0.2685953 , 0.12704062, 0.78305066],
        [0.39286852, 0.31138176, 0.45753765, 0.81801015],
        [0.09792355, 0.40828916, 0.12523672, 0.5531248 ],
        [0.880611  , 0.43022144, 0.9264453 , 0.44643533],
        [0.83197343, 0.        , 0.9023824 , 0.86671305],
        [0.        , 0.        , 1.        , 1.        ],
        [0.3410214 , 0.        , 1.        , 0.55943286],
        [0.        , 0.        , 1.        , 0.53002894],
        [0.86324406, 0.        , 0.9349141 , 1.        ],
        [0.8453997 , 0.3410413 , 0.8932963 , 0.35142574],
        [0.        , 0.04323879, 1.        , 0.5569463 ],
        [0.21614219, 0.33269292, 0.23323284, 0.64280933],
        [0.25019065, 0.36368886, 0.26408157, 0.53905857],
        [0.5902128 , 0.157224  , 0.7901875 , 0.87762386],
        [0.33576405, 0.41885132, 0.6763

In [21]:
model.save_weights(current_directory+'/weights_functional_one_class/original_model_one_class')